**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,6ac2488f401e05799558f7a6015bf03c5354d622,8ac089148362cd529ebbb3d1f1fba4ca368d94478c0e83...,2016-06-08 23:45:00+00:00,2016-06-09 00:00:00+00:00,705,2.3,NaN,NaN,NaN,NaN,...,0.0,7.00,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
1,85dfe100e44ed32347972e7eccc0f5fb90f86146,8ac089148362cd529ebbb3d1f1fba4ca368d94478c0e83...,2016-06-09 00:45:00+00:00,2016-06-09 00:45:00+00:00,512,3.4,NaN,NaN,NaN,NaN,...,0.0,9.20,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
2,cc7852078d7422dfd7764a9276fcd239209976d0,8ac089148362cd529ebbb3d1f1fba4ca368d94478c0e83...,2016-06-09 04:00:00+00:00,2016-06-09 04:15:00+00:00,493,3.2,NaN,NaN,NaN,NaN,...,0.0,8.60,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
3,9bfaddca9732699f6fde5617a3cb0091ba580135,8ac089148362cd529ebbb3d1f1fba4ca368d94478c0e83...,2016-06-09 08:45:00+00:00,2016-06-09 09:00:00+00:00,1390,4.8,NaN,NaN,NaN,NaN,...,0.0,17.20,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
4,3392da9b0dabb6f9f59832ec82944c64968bf117,5fa0a0f93d7c8f729c7f3132eed2de1dcf04fd38e73cee...,2016-06-03 12:45:00+00:00,2016-06-03 12:45:00+00:00,6,0.0,NaN,NaN,NaN,NaN,...,0.0,6.01,Credit Card,303 Taxi,NaN,NaN,None,NaN,NaN,None


In [3]:
client.list_rows(table, max_results=5).to_dataframe().info()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   unique_key              5 non-null      object             
 1   taxi_id                 5 non-null      object             
 2   trip_start_timestamp    5 non-null      datetime64[ns, UTC]
 3   trip_end_timestamp      5 non-null      datetime64[ns, UTC]
 4   trip_seconds            5 non-null      int64              
 5   trip_miles              5 non-null      float64            
 6   pickup_census_tract     0 non-null      float64            
 7   dropoff_census_tract    0 non-null      float64            
 8   pickup_community_area   0 non-null      float64            
 9   dropoff_community_area  0 non-null      float64            
 10  fare                    5 non-null      float64            
 11  tips                    5 non-null      float64  

# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to December 31, 2017.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 15 days and the following 15 days, as long as the days fit within the two-year time frame.  For instance, when calculating the value in this column for January 5, 2016, the window will include the number of trips for the preceding 4 days, the current date, and the following 15 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) micro-course.

In [4]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2018-01-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,trip_date,avg_num_trips
0,2016-01-01,80461.937500
1,2016-01-02,80150.647059
2,2016-01-03,79419.611111
3,2016-01-04,79810.421053
4,2016-01-05,80293.900000


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [5]:
# Lines below will give you a hint or solution code
# q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [6]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() 
                        OVER (
                              PARTITION BY pickup_community_area
                              ORDER BY trip_start_timestamp
                              ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2017-05-01'
                    """

# Check your answer
q_2.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,53.0,2017-05-01 08:15:00+00:00,2017-05-01 08:15:00+00:00,1
1,53.0,2017-05-01 08:45:00+00:00,2017-05-01 09:15:00+00:00,2
2,53.0,2017-05-01 11:45:00+00:00,2017-05-01 12:15:00+00:00,3
3,37.0,2017-05-01 10:30:00+00:00,2017-05-01 10:45:00+00:00,1
4,37.0,2017-05-01 14:00:00+00:00,2017-05-01 14:15:00+00:00,2


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [7]:
# Lines below will give you a hint or solution code
# q_2.hint()
# q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [8]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1)
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                   """

# Check your answer
q_3.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,00e34f77f4495a6128d5b3312099c8f2f4ea25c2650e83...,2017-05-01 01:00:00+00:00,2017-05-01 01:30:00+00:00,NaN
1,00e34f77f4495a6128d5b3312099c8f2f4ea25c2650e83...,2017-05-01 02:15:00+00:00,2017-05-01 02:30:00+00:00,45.0
2,00e34f77f4495a6128d5b3312099c8f2f4ea25c2650e83...,2017-05-01 02:30:00+00:00,2017-05-01 02:45:00+00:00,0.0
3,1721f8d69b5f831e0c16c51510786ca43aaa367d23bec8...,2017-05-01 05:00:00+00:00,2017-05-01 05:15:00+00:00,NaN
4,1721f8d69b5f831e0c16c51510786ca43aaa367d23bec8...,2017-05-01 05:45:00+00:00,2017-05-01 06:15:00+00:00,30.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [9]:
# Lines below will give you a hint or solution code
# q_3.hint()
# q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*